In [1]:
import game
import time
import copy
import torch
import pygame
import random
import constants
import numpy as np
from numba import jit, njit

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
ui = game.GameUI()
ui.update_ui()

init random seed to 2048


In [3]:
g = ui.game

In [15]:
random.seed(2048)

In [16]:
g.restart()
ui.update_ui()

init random seed to 2048


In [4]:
%%time
while not g.get_is_done():
    g.move(direction=None, inplace=True)
    ui.update_ui()

CPU times: user 2.57 s, sys: 237 ms, total: 2.81 s
Wall time: 7.36 s


In [4]:
def random_ai_move(game_:game.Game, ui: game.GameUI=None, searches_per_move=10, search_length=10):
    scores = np.zeros(4)
    init_game_clone = game_.clone()
    for first_move_i in range(4):
        game_clone1 = init_game_clone.clone()
        if ui is not None:
            ui.set_game(game_clone1)
            ui.update_ui()
        first_move = constants.ARROW_KEYS[first_move_i]
        matrix, score, changed = game_clone1.move(direction=first_move, inplace=True)
        if ui is not None:
            ui.set_game(game_clone1)
            ui.update_ui()
        if changed:
            scores[first_move_i] += score
        else:
            continue
        max_cumulative_score = 0
        for later_moves in range(searches_per_move):
            move_number = 1
            game_clone2 = game_clone1.clone()
            changed = True
            score_cumulative = 0
            while changed and move_number < search_length:
                matrix, score, changed = game_clone2.move(inplace=True) # make a random move
                if ui:
                    ui.set_game(game_clone2)
                    ui.update_ui()
                if changed:
                    score_cumulative += score
                    scores[first_move_i] += score
                    move_number += 1
            max_cumulative_score = max(max_cumulative_score, score_cumulative)
        # scores[first_move_i] += max_cumulative_score
    best_move_index = np.argmax(scores)
    best_move = constants.ARROW_KEYS[best_move_index]
    return best_move    

In [29]:
t_0 = time.time()
time.sleep(2)
t = time.time() - t_0
print(round(t, 2))
"{:.2f}".format(t)

2.0


'2.00'

In [7]:
g.set_seed(10)
g.restart()
print(g.seed)
g.get_matrix()
ui.update_ui()

set random seed to 10
init random seed to 10
10


In [10]:
%%time
seed_performance_dict = {}
for seed in range(100):
    print(f"seed {seed}")
    t_0 = time.time()
    g.set_seed(seed)
    g.restart()
    np.random.seed(seed)
    random.seed(seed)
    ui.update_ui()
    for i in range(1500):
        move = random_ai_move(game_=g, searches_per_move=20, search_length=10)
        g.move(move)
        ui.update_ui()
        if g.get_is_done():
            max_val = g.get_max_val()
            print(f"done, {i} iters, score: {g.get_score()}, {round(time.time() - t_0, 2)}s, max_val: {max_val}")
            ui.update_ui()
            seed_performance_dict[seed] = max_val
        if g.has_won():
            print(f"won in {i} moves")
            seed_performance_dict[seed] = max_val
            break

seed 0
set random seed to 0
init random seed to 0
done, 832 iters, score: 14376, 28.68s, max_val: 1024.0
seed 1
set random seed to 1
init random seed to 1
done, 429 iters, score: 6400, 14.91s, max_val: 512.0
seed 2
set random seed to 2
init random seed to 2
done, 678 iters, score: 11672, 23.46s, max_val: 1024.0
seed 3
set random seed to 3
init random seed to 3
done, 887 iters, score: 15092, 30.71s, max_val: 1024.0
seed 4
set random seed to 4
init random seed to 4
done, 421 iters, score: 6084, 14.58s, max_val: 512.0
seed 5
set random seed to 5
init random seed to 5
done, 365 iters, score: 5312, 12.73s, max_val: 512.0
seed 6
set random seed to 6
init random seed to 6
done, 524 iters, score: 7512, 18.12s, max_val: 512.0
seed 7
set random seed to 7
init random seed to 7
done, 865 iters, score: 14664, 29.94s, max_val: 1024.0
seed 8
set random seed to 8
init random seed to 8
done, 410 iters, score: 5760, 14.28s, max_val: 512.0
seed 9
set random seed to 9
init random seed to 9
done, 723 iters

In [11]:
seed_performance_dict

{0: 14376,
 1: 6400,
 2: 11672,
 3: 15092,
 4: 6084,
 5: 5312,
 6: 7512,
 7: 14664,
 8: 5760,
 9: 12168,
 10: 16156,
 11: 3116,
 12: 7308,
 13: 5320,
 14: 12472,
 15: 12456,
 16: 6836,
 17: 6448,
 18: 7336,
 19: 7544,
 20: 6992,
 21: 12096,
 22: 6436,
 23: 6856,
 24: 20348,
 25: 7208,
 26: 15008,
 27: 3188,
 28: 11772,
 29: 7048,
 30: 15876,
 31: 7120,
 32: 16068,
 33: 12004,
 34: 7156,
 35: 6588,
 36: 6860,
 37: 7292,
 38: 14420,
 39: 11348,
 40: 7384,
 41: 14332,
 42: 15400,
 43: 7476,
 44: 11476,
 45: 11924,
 46: 12256,
 47: 2340,
 48: 7028,
 49: 5952,
 50: 3088,
 51: 7200,
 52: 6652,
 53: 7184,
 54: 7012,
 55: 7108,
 56: 7236,
 57: 7192,
 58: 7052,
 59: 7276,
 60: 6568,
 62: 7128,
 63: 14304,
 64: 12188,
 65: 7016,
 66: 13708,
 67: 7064,
 68: 6780,
 69: 15564,
 70: 7396,
 71: 10540,
 72: 1384,
 73: 7376,
 74: 6872,
 75: 7132,
 76: 16240,
 77: 7440,
 78: 7320,
 79: 1432,
 80: 15372,
 81: 7088,
 82: 11660,
 83: 6884,
 84: 12268,
 85: 14220,
 86: 14340,
 87: 7408,
 88: 11596,
 89: 144

In [22]:
%%time
constants.RANDOM_SEED = 2048
g.restart()
ui.update_ui()
for i in range(1500):
    # print(f"i: {i}")
    move = random_ai_move(game_=g, searches_per_move=40, search_length=20)
    g.move(move)
    ui.update_ui()
    if g.get_is_done():
        max_val = g.get_max_val()
        print(f"done, {i} iters, max_val: {max_val}")
        ui.update_ui()
        break
    if g.has_won():
        print(f"won in {i} moves")
        break

init random seed to 2048
done, 663 iters, max_val: 1024
CPU times: user 17.1 s, sys: 298 ms, total: 17.4 s
Wall time: 23.1 s


In [20]:
pygame.quit()

In [31]:
seed_performance_dict

{}

NameError: name 'matrix' is not defined